## 02. Практический кейс. Сбор тренировочных данных.

In [181]:
import json
import multiprocessing
import glob
import os
import numpy as np
import pandas as pd
import re
import requests
import cchardet
import lxml
from bs4 import BeautifulSoup
from tqdm import tqdm
from pprint import pprint

In [182]:
%matplotlib inline
pd.set_option('display.max_columns', None)

### Начнем сбор данных

In [184]:
test_df = pd.read_csv('data/test.csv')

#### Список с брендами/моделями

Создадим список с парами (бренд, модель) для более удобной итерации в процессе обработки

In [186]:
all_brands = list(test_df['brand'].unique())
print(f"all brands: {all_brands}")

# build list with all (brand, model) tuples
brands_models_list = list()

for brand in tqdm(all_brands):
    # select only current brand samples
    brand_samples = test_df[test_df['brand'] == brand]
    # get all unique model names
    brand_models = list(brand_samples['modelName'].unique())
    # add entries to list
    for model in brand_models:
        brands_models_list.append((brand, model))

100%|██████████| 12/12 [00:00<00:00, 298.68it/s]

all brands: ['SKODA', 'AUDI', 'HONDA', 'VOLVO', 'BMW', 'NISSAN', 'INFINITI', 'MERCEDES', 'TOYOTA', 'LEXUS', 'VOLKSWAGEN', 'MITSUBISHI']


#### Словарь с брендами и их вендорами

Создадим словарь, в котором хранится вендор для каждой модели. Так, мы сможем определить вендор, так как он не указан на странице объявления

In [187]:
# build dictionary with vendor for each brand
brands_vendors_dict = dict()

for brand in tqdm(all_brands):
    # select current vendor samples
    brand_samples = test_df[test_df['brand'] == brand]
    # get all unique vendors
    brand_vendors = list(brand_samples['vendor'].unique())
    # add entry to brands/vendors dictionary
    brands_vendors_dict[brand] = brand_vendors[0]
    
# save to .json file
with open('data/brands_vendors_dict.json', 'w') as file:
    json.dump(brands_vendors_dict, file)

100%|██████████| 12/12 [00:00<00:00, 269.77it/s]


#### Автоматизируем процесс добычи ссылок на объявления

Я решил пойти следующим путем: в результатах поиска для определенного бренда/модели мы получаем список с объявлениями. Страниц может быть несколько, а может быть одна. Для определения кол-ва страниц, напишем специальный метод, который находит специфичные теги с указанными классами (определены мною в результате изучения HTML кода страниц) и определяет данным образом кол-во страниц. 

Затем, мы получаем шапку для каждого объявления ссылку на страницу с деталями.  На данном этапе соберем список из ссылок для каждого бренда/модели авто.

Используем для GET запросов библиотеку requests. Затем, обработаем полученный ответ с помощью BeautifulSoup4

In [6]:
base_url = 'https://auto.ru/cars'

In [7]:
# pagination class
pagination_class = 'ListingPagination-module__pages' # span
# listing item link class
item_link_class = 'ListingItemTitle-module__link' # a

In [8]:
def get_pages_number(soup):
    # search for pagination element
    pagination = soup.find('span', class_=pagination_class)
    
    # return 1 if pagination element is not found
    if pagination is None:
        return 1
    
    # find <a> children, i.e. links
    links = pagination.find_all('a')
    
    # return text of last link (converted to integer)
    return int(links[-1].get_text())

In [9]:
def get_item_links(soup):
    # search for item listing links
    item_links = soup.find_all('a', class_=item_link_class)
    # extract links only, i.e. href only
    item_links = [link['href'] for link in item_links]
    
    return item_links

In [14]:
def parse_one_page(brand, model, page):
    # build page query; empty if page is 1 and ?page=n otherwise
    page_query = '' if page == 1 else f'/?page={page}'
    # build request URL
    request_url = f'{base_url}/{brand}/{model}/all{page_query}'
    # send GET request
    try:
        response = requests.get(request_url, timeout=3)
    except requests.exceptions.Timeout:
        return None
    except requests.exceptions.RequestException:
        return None
        
    # set response encoding to UTF-8; removes gibberish characters in response text
    response.encoding = 'utf-8'
    # parse response text with bs4
    soup = BeautifulSoup(response.text, 'lxml')
    
    return soup

In [15]:
def parse_all_item_links(brands_models_list):
    # iterate through (brand, model) tuples
    for (brand, model) in tqdm(brands_models_list):
        # transform to lowercase
        brand = brand.lower()
        model = model.lower()
        print(f"brand: {brand}, model: {model}")
        
        # create directory for brand
        os.makedirs(os.path.join('data', 'links', brand), exist_ok=True)
        # build path for JSON file
        json_path = os.path.join('data', 'links', brand, f'{model}.json')
        
        if os.path.exists(json_path):
            print("skipping...")
            continue
        
        # build list of links for current model
        model_links_list = list()
        
        # parse first page
        first_page_soup = parse_one_page(brand, model, 1)
        if first_page_soup is None:
            print("\tskipping page 1 due to connection timeout/error")
            continue
        # get number of pages
        pages_number = get_pages_number(first_page_soup)
        # add links from first page
        model_links_list.extend(get_item_links(first_page_soup))
        print(f"\tprocessed page 1/{pages_number} ({len(model_links_list)} items)")
        
        # continue if more than 1 page
        if pages_number > 1:
            for i in range(2, pages_number + 1):
                page_soup = parse_one_page(brand, model, i)
                if page_soup is None:
                    print(f"\tskipping page {i} due to connection timeout/error")
                    continue
                # add links from page
                model_links_list.extend(get_item_links(page_soup))
                print(f"\tprocessed page {i}/{pages_number} ({len(model_links_list)} items)")
            
        # just in case: save list to JSON on each iteration
        with open(json_path, 'w') as file:
            json.dump(model_links_list, file)

In [ ]:
parse_all_item_links(brands_models_list)

#### Посчитаем сколько мы собрали объявлений

In [302]:
total_links = 0
links_by_brand = dict()
links_by_brand_model = dict()

# iterate through all .json files
for model_path in glob.glob(os.path.join('data', 'links', '*', '*.json')):
    # infer brand and model from path
    brand = os.path.basename(os.path.dirname(model_path))
    model = os.path.basename(model_path)[:-5]
    
    # open .json file
    with open(model_path, 'r') as file:
        json_file = json.load(file)
        
    # count number of items in array
    num_links = len(json_file)
    
    # 1) add up to total number of items
    total_links += num_links
    
    # 2) add up to number of items by brand
    if brand in links_by_brand:
        links_by_brand[brand] += num_links
    else:
        links_by_brand[brand] = num_links
        
    # 3) add up to number of items by brand/model
    if brand in links_by_brand_model:
        # add up to number of model items
        links_by_brand_model[brand][model] = num_links
    else:
        links_by_brand_model[brand] = dict()
        links_by_brand_model[brand][model] = num_links

In [303]:
print(f"total number of links collected: {total_links}")
print("number of links by brand")
pprint(links_by_brand)
# pprint(links_by_brand_model)

# save both dictionaries
with open('data/links_by_brand.json', 'w') as file:
    json.dump(links_by_brand, file)
with open('data/links_by_brand_model.json', 'w') as file:
    json.dump(links_by_brand_model, file)

total number of links collected: 145099
number of links by brand
{'AUDI': 10677,
 'BMW': 15337,
 'HONDA': 5804,
 'INFINITI': 1909,
 'LEXUS': 3381,
 'MERCEDES': 20726,
 'MITSUBISHI': 11909,
 'NISSAN': 20596,
 'SKODA': 9158,
 'TOYOTA': 23596,
 'VOLKSWAGEN': 18591,
 'VOLVO': 3415}


#### Достаем данные об автомобиле из объявления

Теперь раз мы имеем ссылки на объявления, нужно будет доставать информацию об автомобиле. Для этого, изучим страницы объявления и найдем нужные HTML элементы и их соответсвующие классы. Надо еще учесть, что при запросах могут возникать ошибки (объявление удалено или ссылка неправильная/недействительна). 

Затем, снова запустим этот процесс на нескольких девайсах (желательно) и соберем данные в .csv файл

In [279]:
car_url = 'https://auto.ru/cars/used/sale/volkswagen/touareg/1101820126-ed5d7941/'

In [280]:
"""
Simple builder of atribute HTML class
"""
def get_atribute_classname(atribute):
    return f'CardInfoRow_{atribute}'

In [281]:
"""
Convert string into integer
"""
def convert_string_to_int(s):
    value = ''.join([c for c in s if c.isdigit()])
    return int(value)

In [282]:
"""
Splits engine information string into three atributes
"""
def get_engine_characteristics(engine_atributes):
    # split string by /
    splitted = engine_atributes.split('/')
    # extract each engine characteristic
    engine_volume = splitted[0].split()[0]
    engine_power = ''.join([c for c in splitted[1].split()[0] if c.isdigit()])
    fuel_type = splitted[2].strip().lower()
    
    return float(engine_volume), float(int(engine_power)), fuel_type

In [283]:
"""
Retrieve page and parse using lxml parser
"""
def get_car_page(car_url, timeout=3):
    try:
        response = requests.get(car_url, timeout=timeout)
    except requests.exceptions.RequestException:
        return None
    
    # check response status code
    if response.status_code != 200:
        return None
    
    # set response encoding to UTF-8; removes gibberish characters in response text
    response.encoding = 'utf-8'
    # parse response text with bs4
    soup = BeautifulSoup(response.text, 'lxml')
    
    return soup

In [284]:
body_type_atribute = 'bodytype'
color_atribute = 'color'
engine_atribute = 'engine' # 3.0 л / 249 л.с. / Дизель
mileage_atribute = 'kmAge'
year_atribute = 'year'
transmission_atribute = 'transmission'
owners_count_atribute = 'ownersCount'
pts_atribute = 'pts'
drive_atribute = 'drive'
wheel_atribute = 'wheel'
# price element classname
price_classname = 'OfferPriceCaption__price' # span

# extra
# condition_atribute = 'state'
# owning_time_atribute = 'owningTime'
# customs_atribute = 'customs'

In [285]:
all_atributes = [body_type_atribute, color_atribute, engine_atribute, mileage_atribute, year_atribute, 
                transmission_atribute, owners_count_atribute, pts_atribute, drive_atribute, wheel_atribute]

In [286]:
atribute_to_feature_mapping = {
    'bodytype': 'bodyType',
    'kmAge': 'mileage',
    'transmission': 'vehicleTransmission',
    'ownersCount': 'Владельцы',
    'pts': 'ПТС',
    'drive': 'Привод',
    'wheel': 'Руль'
}

In [287]:
"""
Combine previous functions and extract atribute values
"""
def get_car_info(car_url, brand, model, vendor):
    # retrieve and parse car page
    car_page = get_car_page(car_url)
    # build car info dictionary
    info_dict = dict()
    info_dict['brand'] = brand
    info_dict['model_name'] = model
    info_dict['vendor'] = vendor
    
    # extract price
    price_element = car_page.find('span', class_=price_classname)
    # add price to dictionary
    info_dict['price'] = convert_string_to_int(price_element.get_text())
    
    # extract atributes
    for atribute in all_atributes:
        # find atribute element
        atribute_element = car_page.find('li', class_=get_atribute_classname(atribute))
        
        if atribute_element:
            # get its <span> children (one has atribute name and other has value)
            atribute_element_span_children = atribute_element.find_all('span')
            # extract atribute value
            atribute_value = atribute_element_span_children[-1].get_text().replace('\xa0', ' ')
            
            # split engine characteristics
            if atribute == engine_atribute:
                engine_volume, engine_power, fuel_type = get_engine_characteristics(atribute_value)
                info_dict['engineDisplacement'] = engine_volume
                info_dict['enginePower'] = engine_power
                info_dict['fuelType'] = fuel_type
                continue
                
            # modify mileage value
            if atribute == mileage_atribute:
                atribute_value = convert_string_to_int(atribute_value)
            
            # update car info dictionary
            if atribute in atribute_to_feature_mapping.keys():
                info_dict[atribute_to_feature_mapping[atribute]] = atribute_value
            else:
                info_dict[atribute] = atribute_value
        
    return info_dict

In [288]:
get_car_info(car_url, 'VOLKSWAGEN', 'TOUAREG', brands_vendors_dict['VOLKSWAGEN'])

{'brand': 'VOLKSWAGEN',
 'model_name': 'TOUAREG',
 'vendor': 'EUROPEAN',
 'price': 4270000,
 'bodyType': 'внедорожник 5 дв.',
 'color': 'чёрный',
 'engineDisplacement': 3.0,
 'enginePower': 249.0,
 'fuelType': 'дизель',
 'mileage': 22449,
 'year': '2018',
 'vehicleTransmission': 'автоматическая',
 'Владельцы': '1 владелец',
 'ПТС': 'Оригинал',
 'Привод': 'полный',
 'Руль': 'Левый'}

#### Дополнительно: достаем данные о характеристике модели из каталога auto.ru

In [55]:
response = requests.get("https://auto.ru/catalog/cars/bmw/x4/21203948/21204042/specifications/21204042_21485476_21204195/")
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, 'html.parser')

In [57]:
element = soup.find('dt', class_='list-values__label', string='Количество дверей')

In [61]:
element.find_next_sibling('dd')

<dd class="list-values__value">5</dd>

In [35]:
soup.find_all('dt', string='Количество дверей')

[]

### Собираем все воедино

#### Важное обновление признаков

Я решил избавиться от признаков numberOfDoors и productionDate в силу того, что их сложно достать для тренировочных данных. numberOfDoors необходимо доставать из каталога, что в принципе возможно, но пока отложим в сторону.

#### Собираем датафрейм из JSON файлов

In [6]:
def fill_df_entry(car):
    car_info = dict()
    
    for col in REQUIRED_COLUMNS:
        if col in car:
            car_info[col] = car[col]
        else:
            car_info[col] = np.nan
            
    return car_info

In [7]:
# list of columns which should be present in each car listing
REQUIRED_COLUMNS = ['car_url', 'brand', 'model_name', 'vendor', 'price', 'bodyType', 'color', 'engineDisplacement',
                   'enginePower', 'fuelType', 'mileage', 'year', 'vehicleTransmission', 'Владельцы', 'ПТС',
                   'Привод', 'Руль']

In [12]:
def create_df_for_brand(brand):
    # create dataframe
    df = pd.DataFrame(columns=REQUIRED_COLUMNS)
    # get list of .json file paths
    model_paths = glob.glob(os.path.join('data/cars', brand, '*.json'))
    
    # iterate through each model .json file
    for model_path in model_paths:
        # open .json file with list of car listings of current model
        with open(model_path, 'r') as file:
            cars_list = json.load(file)

        # iterate through each car listing
        for car in cars_list:
            # append car info to dataframe
            df = df.append(fill_df_entry(car), ignore_index=True)
            
    # save dataframe
    df.to_csv(f'data/dataframes/{brand}.csv', index=None)

In [17]:
# let's use multiprocessing features
# start 12 processes with each one running above function
for brand in all_brands:
    p = multiprocessing.Process(target=create_df_for_brand, args=(brand,))
    p.start()
    print(f"started process for {brand}")

started process for SKODA
started process for AUDI
started process for HONDA
started process for VOLVO
started process for BMW
started process for NISSAN
started process for INFINITI
started process for MERCEDES
started process for TOYOTA
started process for LEXUS
started process for VOLKSWAGEN
started process for MITSUBISHI


In [8]:
# create dataframe
df = pd.DataFrame(columns=REQUIRED_COLUMNS)

for brand in all_brands:
    # open brand .csv file
    brand_df = pd.read_csv(f'data/dataframes/{brand}.csv')
    # append to main dataframe
    df = df.append(brand_df)
    
    # save dataframe
    df.to_csv(f'data/train_combined.csv', index=None)

### Почистим собранные данные

In [170]:
train_df = pd.read_csv('data/train_combined.csv')

#### Удаляем строки с пустыми значениями

In [171]:
# drop rows with missing values
train_df = train_df.dropna()

#### Удаляем повторяющиеся значения

In [172]:
# drop duplicates
train_df = train_df.drop_duplicates()

#### bodyType

Удалим автомобили с типом кузова, которого нет в тестовых данных

In [173]:
train_body_types = list(train_df['bodyType'].unique())
test_body_types = list(test_df['bodyType'].unique())

not_matched_body_types = []
for body_type in train_body_types:
    if body_type not in test_body_types:
        not_matched_body_types.append(body_type)

In [174]:
train_df = train_df[~train_df['bodyType'].isin(not_matched_body_types)]

#### engineDisplacement

У некоторых авто объем двигателя гораздо больше других. Это автомобили с электродвигателем, поэтому поставим значение engineDisplacement на 0.0

In [175]:
train_df.loc[(train_df['engineDisplacement'] > 7.0), 'engineDisplacement'] = 0.0

#### fuelType

In [176]:
# get test dataframe fuel types
fuel_types = list(test_df['fuelType'].unique())
# leave only rows with one of fuel types from list above
train_df = train_df[train_df['fuelType'].isin(fuel_types)]

#### productionDate и mileage

Нужно поменять признак year на productionDate, а также перевести его из float в int. Также перевести mileage в int

In [177]:
# rename 'year' to 'productionDate'
train_df.rename(columns={'year': 'productionDate'}, inplace=True)
# convert float to integer
train_df.loc[:, 'productionDate'] = train_df['productionDate'].astype('int')
train_df.loc[:, 'mileage'] = train_df['mileage'].astype('int')

#### Приводим к общим колонкам

In [178]:
columns = list(test_df.columns)
columns.remove('modelDate')
columns.remove('numberOfDoors')
columns.remove('sell_id')
# columns.append('car_url')
columns.append('price')

In [179]:
train_df = train_df[columns]

In [180]:
# save .csv file
train_df.to_csv('data/train.csv', index=None)

### Используем найденные на Kaggle данные для тренировки

In [163]:
# load train and test data
test_df = pd.read_csv('data/test_modified.csv')
all_auto_ru_09_09_2020 = pd.read_csv('data/all_auto_ru_09_09_2020.csv')

In [164]:
# display both dataframes
display(test_df.head())
display(all_auto_ru_09_09_2020.head())

,bodyType,brand,color,engineDisplacement,enginePower,fuelType,mileage,modelDate,model_name,numberOfDoors,productionDate,sell_id,vehicleTransmission,vendor,Владельцы,ПТС,Привод,Руль
0,лифтбек,SKODA,синий,1.2,105.0,бензин,74000,2013,OCTAVIA,5,2014,1100575026,роботизированная,EUROPEAN,3 или более,Оригинал,передний,Левый
1,лифтбек,SKODA,чёрный,1.6,110.0,бензин,60563,2017,OCTAVIA,5,2017,1100549428,механическая,EUROPEAN,1 владелец,Оригинал,передний,Левый
2,лифтбек,SKODA,серый,1.8,152.0,бензин,88000,2013,SUPERB,5,2014,1100658222,роботизированная,EUROPEAN,1 владелец,Оригинал,передний,Левый
3,лифтбек,SKODA,коричневый,1.6,110.0,бензин,95000,2013,OCTAVIA,5,2014,1100937408,автоматическая,EUROPEAN,1 владелец,Оригинал,передний,Левый
4,лифтбек,SKODA,белый,1.8,152.0,бензин,58536,2008,OCTAVIA,5,2012,1101037972,автоматическая,EUROPEAN,1 владелец,Оригинал,передний,Левый


,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,engineDisplacement,enginePower,description,mileage,Комплектация,Привод,Руль,Состояние,Владельцы,ПТС,Таможня,Владение,price,start_date,hidden,model
0,Седан,AUDI,040001,бензин,1990.0,2.8 MT (174 л.с.) 4WD,4.0,1991,SEDAN MECHANICAL 2.8,MECHANICAL,2.8,174.0,"Машина в приличном состоянии ,не гнилая не р...",350000,{'id': '0'},полный,LEFT,NaN,3.0,ORIGINAL,True,NaN,200000.0,2019-10-03T08:09:11Z,NaN,100
1,Седан,AUDI,EE1D19,бензин,1982.0,1.8 MT (90 л.с.),4.0,1986,SEDAN MECHANICAL 1.8,MECHANICAL,1.8,90.0,Машина в оригинале не гнилая все вопросы по те...,173424,{'id': '0'},передний,LEFT,NaN,3.0,ORIGINAL,True,NaN,60000.0,2020-09-06T06:49:40Z,NaN,100
2,Универсал 5 дв.,AUDI,0000CC,бензин,1988.0,2.3 MT (136 л.с.) 4WD,5.0,1989,WAGON_5_DOORS MECHANICAL 2.3,MECHANICAL,2.3,136.0,ПТС Оригинал!\nПолный комплект ключей!\nПо рез...,230000,{'id': '0'},полный,LEFT,NaN,3.0,ORIGINAL,True,NaN,99000.0,2020-09-02T14:04:21Z,NaN,100
3,Седан,AUDI,CACECB,бензин,1988.0,1.8 MT (90 л.с.),4.0,1989,SEDAN MECHANICAL 1.8,MECHANICAL,1.8,90.0,NaN,240000,{'id': '0'},передний,LEFT,NaN,3.0,ORIGINAL,True,NaN,65000.0,2020-08-23T17:40:09Z,NaN,100
4,Седан,AUDI,040001,бензин,1990.0,2.0 MT (101 л.с.),4.0,1991,SEDAN MECHANICAL 2.0,MECHANICAL,2.0,101.0,"Машина не гнилая, дыр нет, днище целое, даже в...",300000,{'id': '0'},передний,LEFT,NaN,3.0,DUPLICATE,True,NaN,100000.0,2020-09-08T09:22:07Z,NaN,100


#### price

Удаляем авто без указанной цены

In [165]:
print(f"initially we had: {len(all_auto_ru_09_09_2020)}")

train_df = all_auto_ru_09_09_2020.dropna(axis=0, subset=['price'])

print(f"after removing NaN price samples we have: {len(train_df)}")

initially we had: 89378
after removing NaN price samples we have: 88968


#### bodyType

Удаляем те автомобили, у которых тип кузова не из списка кузовов тестовых сэмплов

In [166]:
print(f"before we had: {len(train_df)}")

# lowercase whole bodyType column in train dataframe
train_df['bodyType'] = train_df['bodyType'].str.lower()

test_df_body_types = list(test_df['bodyType'].unique())
train_df = train_df[train_df['bodyType'].isin(test_df_body_types)]

print(f"after preprocessing by bodyType we have: {len(train_df)}")

before we had: 88968
after preprocessing by bodyType we have: 85143


/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


#### brand

Удаляем автомобили, бренда которых нет в списке брендов тестовых сэмплов

In [157]:
print(f"before we had: {len(train_df)}")

test_df_brands = list(test_df['brand'].unique())
train_df = train_df[train_df['brand'].isin(test_df_brands)]

print(f"after preprocessing by brand we have: {len(train_df)}")

before we had: 85143
after preprocessing by brand we have: 46645


#### color

Используя словарь значений, найденный в Kaggle ноутбуке, конвертируем цвета

In [168]:
# color dictionary from Kaggle notebook
dict_color = {'040001':'чёрный', 'EE1D19':'красный', '0000CC':'синий', 
              'CACECB':'серебристый', '007F00':'зелёный', 'FAFBFB':'белый', 
              '97948F':'серый', '22A0F8':'голубой', '660099':'пурпурный', 
              '200204':'коричневый', 'C49648':'бежевый', 'DEA522':'золотистый', 
              '4A2197':'фиолетовый', 'FFD600':'жёлтый', 'FF8649':'оранжевый', 
              'FFC0CB':'розовый'}

train_df['color'] = train_df['color'].map(dict_color)

#### engineDisplacement

Некоторые значения содержат в себе нечисленные символы, поэтому тип данных в данной колонке указан как object. Нужно избавить от таких значений и конвертировать тип колонки в float.

In [169]:
print(f"before we had: {len(train_df)}")

# first, remove samples which do not match specified pattern
train_df = train_df[train_df['engineDisplacement'].str.match('(.)\.(.*)')]

# then, we need to remove letters
engine_displacements_list = list(train_df['engineDisplacement'].unique())
dict_engine_displacement = dict()
for s in engine_displacements_list:
    dict_engine_displacement[s] = re.sub('[^0-9|\.]', '', s)
train_df['engineDisplacement'] = train_df['engineDisplacement'].map(dict_engine_displacement)
    
# convert to float
train_df['engineDisplacement'] = train_df['engineDisplacement'].astype('float')

print(f"after preprocessing by engineDisplacement we have: {len(train_df)}")

before we had: 85143
after preprocessing by engineDisplacement we have: 67458


#### modelDate и numberOfDoors

Переводим из float в int

In [170]:
train_df['modelDate'] = train_df['modelDate'].astype('int')
train_df['numberOfDoors'] = train_df['numberOfDoors'].astype('int')

#### vehicleTransmission

Снова используем словарь для перевода значений

In [171]:
# transmission dictionary
dict_transmission = {'AUTOMATIC': 'автоматическая', 
                     'MECHANICAL':'механическая', 
                     'ROBOT':'роботизированная', 
                     'VARIATOR':'вариатор'}

# convert using dictionary
train_df['vehicleTransmission'] = train_df['vehicleTransmission'].map(dict_transmission)

#### vendor

Используем мною созданный словарь бренд/вендор

In [ ]:
with open('data/brands_vendors_dict.json', 'r') as file:
    brands_vendors_dict = json.load(file)
    
train_df['vendor'] = train_df['brand'].map(lambda x: brands_vendors_dict[x])

#### Владельцы

Используем словарь

In [173]:
dict_owners = {4.0:'3 или более', 3.0: '3 или более', 2.0: '2 владельца', 1.0: '1 владелец'}

train_df['Владельцы'] = train_df['Владельцы'].map(dict_owners)

#### ПТС

Используем словарь

In [174]:
dict_pts = {'ORIGINAL': 'Оригинал', 'DUPLICATE': 'Дубликат'}

train_df['ПТС'] = train_df['ПТС'].map(dict_pts)

#### Руль

Используем словарь

In [175]:
dict_wheel = {'LEFT': 'Левый', 'RIGHT': 'Правый'}

train_df['Руль'] = train_df['Руль'].map(dict_wheel)

#### Выбираем колонки

Не используем model_name, так как этого признака нет от слова совсем в тренировочных данных. Также, не забыть добавить колонку price

In [186]:
columns = list(test_df.columns)

columns.remove('sell_id')
columns.remove('model_name')
columns.remove('vendor')
columns.append('price')

train_df = train_df[columns]

In [187]:
# save train data
train_df.to_csv('data/train_kaggle.csv', index=None)